In [12]:
import pandas as pd
import numpy as np
import os
from gtda.time_series import TakensEmbedding, SingleTakensEmbedding, SlidingWindow, PearsonDissimilarity
from sklearn.preprocessing import StandardScaler
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_point_cloud
from gtda.diagrams import PersistenceLandscape, BettiCurve, HeatKernel, PersistenceImage, Silhouette, Amplitude, PersistenceEntropy
import matplotlib.pyplot as plt
from gtda.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [7]:
DATA_DIR = 'Financial Data'
scaler = StandardScaler()

crypto = {}
for fn in os.listdir(os.path.join(DATA_DIR, 'cryptocurrencies')):
    crypto[fn] = pd.read_csv(os.path.join(DATA_DIR, 'cryptocurrencies', fn))
    crypto[fn]['Volume'] = crypto[fn]['Volume'].astype(float)
    crypto[fn].drop(columns=['Year'], inplace=True)
    crypto[fn].iloc[:, 1:] = scaler.fit_transform(crypto[fn].iloc[:, 1:])

currency = {}
for fn in os.listdir(os.path.join(DATA_DIR, 'currencies')):
    currency[fn] = pd.read_csv(os.path.join(DATA_DIR, 'currencies', fn))
    currency[fn].drop(columns=['Year', 'Volume'], inplace=True)
    currency[fn].iloc[:, 1:] = scaler.fit_transform(currency[fn].iloc[:, 1:])

stock = {}
for fn in os.listdir(os.path.join(DATA_DIR, 'stocks')):
    stock[fn] = pd.read_csv(os.path.join(DATA_DIR, 'stocks', fn))
    stock[fn]['Volume'] = stock[fn]['Volume'].astype(float)
    stock[fn].drop(columns=['Symbol', 'Date'], inplace=True)
    stock[fn].iloc[:, 1:] = scaler.fit_transform(stock[fn].iloc[:, 1:])

In [8]:
X = stock['AAPL.csv'].drop(columns=['Close']).values
y = stock['AAPL.csv']['Close'].values

In [9]:
SW = SlidingWindow()
# X_sw, yr = SW.fit_transform_resample(X, y)
PD = PearsonDissimilarity()
# X_pd = PD.fit_transform(X_sw)
VR = VietorisRipsPersistence(metric="precomputed")
# X_vr = VR.fit_transform(X_pd)  # "precomputed" required on dissimilarity data
Ampl = Amplitude()
# X_a = Ampl.fit_transform(X_vr)
RFR = RandomForestRegressor()

pipe = make_pipeline(SW, PD, VR, Ampl, RFR)

pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

In [18]:
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

# print(mean_squared_error(y, y_pred))

(array([-0.63561667, -0.34888702, -0.74471036, ...,  1.82478223,
         1.51643905,  1.64774899]),
 0.797337281646092)

In [16]:
y_pred.shape, y.shape

((5137,), (5146,))